### Init

In [646]:
import robotic as ry
import time

try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing1.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.view()

0

### Komo probs

In [647]:
way0 = C.addFrame('way0', 'start_area')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0.2 0.15 0.1) d(90 0 0 0)')

way1 = C.addFrame('way1', 'start_area')

way1.setShape(ry.ST.marker, size=[.2])
way1.setPosition(C.getFrame('l_gripper').getPosition())

C.view()

0

In [648]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-0.08, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductXZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [-1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [649]:
def create_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-0.08, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductXZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [-1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [650]:
def throw_position_komo(C, gripper_name, grasp_frame_name):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way0'], ry.OT.eq, [1e1], [0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret2)
    return komo

In [651]:
def post_throw_position_komo(C, gripper_name, throw_waypoint_frame):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, throw_waypoint_frame], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way1'], ry.OT.eq, [1e1], [0])
    ret3 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret3)
    return komo

In [652]:
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'paddle_pick_point', q0, qHome)
path0 = komo0.getPath()

komo = create_grasp_komo(C, 'l_gripper', 'paddle', q0, qHome)
path = komo.getPath()

komo2 = throw_position_komo(C, 'l_gripper', 'cargo')
path2 = komo2.getPath()

komo3 = post_throw_position_komo(C, 'l_gripper', 'way1')
path3 = komo3.getPath()

{ time: 0.030228, evals: 188, done: 1, feasible: 1, sos: 0.868712, f: 0, ineq: 0, eq: 0.00298392 }
{ time: 0.060266, evals: 327, done: 1, feasible: 1, sos: 0.916311, f: 0, ineq: 0, eq: 0.0059314 }
{ time: 0.000338, evals: 5, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00277823 }
{ time: 0.000259, evals: 4, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.000212083 }


### Pick up ball

In [653]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperCloseGrasp(ry._left, 'paddle') 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

C.getFrame('paddle').unLink()
C.attach('l_gripper', 'paddle')

# print(path2.shape)
# bot.move(path2, [3.])
# while bot.getTimeToEnd() > 0:
#     bot.sync(C, .1)

# pre_throw_pos = C.getFrame('l_gripper').getPosition()

# bot.move(path3, [3.])
# bot.gripperMove(ry._left, width=0.075, speed=5)
# while bot.getTimeToEnd() > 0:
#     bot.sync(C, .1)


# bot.home(C)

### Calculate throw

In [ ]:
import numpy as np
from scipy.optimize import minimize


def calculate_throw_parameters_bin(initial_position, bin_dimensions, gravity=9.81):
   def loss_function(params):
       release_height, v_magnitude, theta_xz = params
       x0, y0, z0 = initial_position
       bottom_left = bin_dimensions['bottom_left']
       top_right = bin_dimensions['top_right']
      
       release_position = (x0, y0, release_height)
      
       v_x = v_magnitude * np.cos(theta_xz)
       v_z = v_magnitude * np.sin(theta_xz)
      
       t_flight = 2 * v_z / gravity
      
       x_landing = x0 + v_x * t_flight
      
       within_x_bounds = bottom_left[0] <= x_landing <= top_right[0]
       within_y_bounds = bottom_left[1] <= y0 <= top_right[1]
      
       if within_x_bounds and within_y_bounds:
           return 0 
       else:
           dx = max(0, bottom_left[0] - x_landing, x_landing - top_right[0])
           dy = max(0, bottom_left[1] - y0, y0 - top_right[1])
           return dx + dy


   initial_guess = [initial_position[2], 5.0, np.pi / 4]
  
   bounds = [(initial_position[2], initial_position[2] + 1.0),
             (1.0, 20.0), 
             (0, np.pi / 2)] 
  
   result = minimize(loss_function, initial_guess, bounds=bounds, method='SLSQP')
   if result.success:
       release_height, v_magnitude, theta_xz = result.x
       v_x = v_magnitude * np.cos(theta_xz)*2
       v_z = v_magnitude * np.sin(theta_xz)*2
       bin_pos = C.getFrame("bin").getPosition()
       x = np.abs(bin_pos[0]-initial_position[0])
       y = np.abs(bin_pos[1]-initial_position[1])
       x_y_dist = np.sqrt(np.square(x) + np.square(y))
       print(f"Bin position {bin_pos} and init_pos: {initial_position}")
       print(f"is positive x: {((bin_pos[0]-initial_position[0])/np.abs(bin_pos[0]-initial_position[0]))}")
       print(f"initial v_x = {v_x}")
       v_y = v_x* (y/x_y_dist) * ((bin_pos[1]-initial_position[1])/np.abs(bin_pos[1]-initial_position[1]))
       v_x = v_x* (x/x_y_dist) * ((bin_pos[0]-initial_position[0])/np.abs(bin_pos[0]-initial_position[0]))
       print(f"After v_x:{v_x} and v_y:{v_y}")
       return {
           'release_position': [initial_position[0], initial_position[1], release_height],
           'velocity': [v_x, v_y, v_z],
           'angle_xz': theta_xz,
       }
   else:
       raise ValueError("Optimization failed to find valid parameters.")


initial_pos = C.getFrame("cargo").getPosition()


bin_dims = {
   'bottom_left': (0.25, 0.25, 0.14),
   'top_right': (0.75, 0.75, 0.24) 
}
result = calculate_throw_parameters_bin(initial_pos, bin_dims)


print("Release Position:", result['release_position'])
print("Velocity:", result['velocity'])
print("Release Angle (radians):", result['angle_xz'])




C.addFrame("release_frame").setPosition(result['release_position']).setShape(ry.ST.marker,[.4]).setColor([1,0,0]).setContact(0)


C.view()
#finding the initial point from the release point and the velocity
#find the backward kinematics and simulate it


def find_initial_point_from_release(release_position, release_velocity, robot_base, height_threshold=0.2, distance_threshold=1.53):
   """
   Finds the initial position for the throwing motion based on the release position and velocity.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.
       height_threshold: Minimum allowable height for the initial position.
       distance_threshold: Maximum allowable distance from the robot base.


   Returns:
       initial_position: The computed initial position, if found.
   """
   # Compute the initial position using backward kinematics
   time_step = 0.01  # Small time step for simulation
   gravity = np.array([0, 0, -9.81])  # Gravity vector


   # Backward integrate to find the initial position
   velocity = np.array(release_velocity)
   position = np.array(release_position)
   distance_to_base = np.linalg.norm(position - np.array(robot_base))
   iteration = 0
   prev_pos = position
   while position[2] > height_threshold and distance_to_base <= distance_threshold:  # Ensure height is above threshold
       iteration+=1
       prev_pos = position
       position -= velocity * time_step
       velocity -= gravity * time_step
       # Check distance to the robot base
       distance_to_base = np.linalg.norm(position - np.array(robot_base))
       #print(f"Iter {iteration}")
   iteration = 0
   velocity = np.array(release_velocity)
   time_step = .1
   position = np.array(release_position)


   while position[2] > height_threshold:
       prev_pos = position
       position += velocity * time_step
       velocity += gravity * time_step
       iteration += 1
       frame = C.addFrame(f"trajectory-{iteration}").setShape(ry.ST.marker,[.2]).setColor([0,0,1]).setPosition(position)
       C.view()
       print(f"Frame {iteration} added with position {frame.getPosition()} new vel:{velocity}")
   return prev_pos  # No valid initial position found






def velocity_to_rotation_matrix(velocity_vector):
   """
   Construct a rotation matrix where the y-axis aligns with the given velocity vector.


   Args:
       velocity_vector (np.ndarray): Velocity vector in world frame.


   Returns:
       np.ndarray: 3x3 rotation matrix.
   """
   # Normalize the velocity vector to get the y-axis
   y_axis = velocity_vector / np.linalg.norm(velocity_vector)


   # Define a reference z-axis (world z-axis)
   z_axis_world = np.array([0, 0, 1])


   # Compute the x-axis using the cross product of z and y
   x_axis = np.cross(z_axis_world, y_axis)


   # Handle the case where the velocity vector is parallel to the z-axis
   if np.linalg.norm(x_axis) < 1e-6:
       z_axis_world = np.array([1, 0, 0])  # Choose x-axis as reference
       x_axis = np.cross(z_axis_world, y_axis)


   x_axis /= np.linalg.norm(x_axis)  # Normalize the x-axis


   # Recompute the z-axis to ensure orthogonality
   z_axis = np.cross(y_axis, x_axis)
   z_axis /= np.linalg.norm(z_axis)


   # Construct the rotation matrix
   rotation_matrix = np.column_stack((x_axis, y_axis, z_axis))
   return rotation_matrix




def rotation_matrix_to_quaternion(rotation_matrix):
   """
   Converts a 3x3 rotation matrix to a quaternion.
  
   Args:
       rotation_matrix (numpy.ndarray): 3x3 rotation matrix.
      
   Returns:
       numpy.ndarray: Quaternion [w, x, y, z].
   """
   R = rotation_matrix
   trace = np.trace(R)


   if trace > 0:
       S = np.sqrt(trace + 1.0) * 2  # S=4*qw
       w = 0.25 * S
       x = (R[2, 1] - R[1, 2]) / S
       y = (R[0, 2] - R[2, 0]) / S
       z = (R[1, 0] - R[0, 1]) / S
   elif (R[0, 0] > R[1, 1]) and (R[0, 0] > R[2, 2]):
       S = np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2]) * 2  # S=4*qx
       w = (R[2, 1] - R[1, 2]) / S
       x = 0.25 * S
       y = (R[0, 1] + R[1, 0]) / S
       z = (R[0, 2] + R[2, 0]) / S
   elif R[1, 1] > R[2, 2]:
       S = np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2]) * 2  # S=4*qy
       w = (R[0, 2] - R[2, 0]) / S
       x = (R[0, 1] + R[1, 0]) / S
       y = 0.25 * S
       z = (R[1, 2] + R[2, 1]) / S
   else:
       S = np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1]) * 2  # S=4*qz
       w = (R[1, 0] - R[0, 1]) / S
       x = (R[0, 2] + R[2, 0]) / S
       y = (R[1, 2] + R[2, 1]) / S
       z = 0.25 * S


   return np.array([w, x, y, z])
# Example usage
velocity = np.array([1, 2, 3])  # Example velocity vector
rotation_matrix = velocity_to_rotation_matrix(velocity)


print("Rotation Matrix:")
print(rotation_matrix)
def pick_last_object_if_valid(release_position, release_velocity, robot_base):
   """
   Simulates the robot to pick the last object if conditions are met.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.


   Returns:
       None
   """
   # Find the initial position
   initial_position = find_initial_point_from_release(release_position, release_velocity, robot_base)


   if initial_position is not None:
       print(f"Initial Position Found: {initial_position}")
       # Visualize the initial position
       C.addFrame("initial_position").setPosition(initial_position).setShape(ry.ST.marker, [0.4]).setContact(0).setColor([0,1,0]).setQuaternion(rotation_matrix_to_quaternion(velocity_to_rotation_matrix(release_velocity)))
       C.view()
       return initial_position
       # Simulate the robot to pick the last object
       # Add your grasping or motion planning logic here
   else:
       print("No valid initial position found.")


initial_pos = pick_last_object_if_valid(np.array(result['release_position']),np.array(result["velocity"]),C.getFrame("l_panda_base").getPosition())


#go to inital position with y axis being scalar to the velocity vector
def goto_init(gripper_name, initial_pos_name):
   komo = ry.KOMO(C, 1, 1, 0, True)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [0, 0, 0])
   komo.addObjective([1], ry.FS.scalarProductYZ, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [-1])
   ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
   print(ret2)
   return komo






Bin position [1.5  2.   0.08] and init_pos: [0.65767401 2.0022912  0.48670724]
is positive x: 1.0
initial v_x = 0.7071048987372022
After v_x:0.707102282856048 and v_y:-0.0019233818951248532
Release Position: [np.float64(0.6576740145683289), np.float64(2.002291202545166), np.float64(0.48670724034309387)]
Velocity: [np.float64(0.707102282856048), np.float64(-0.0019233818951248532), np.float64(7.071086636308822)]
Release Angle (radians): 0.7854008255792999
Rotation Matrix:
[[-0.89442719  0.26726124 -0.35856858]
 [ 0.4472136   0.53452248 -0.71713717]
 [ 0.          0.80178373  0.5976143 ]]
Frame 1 added with position [0.72838424 2.00209886 1.1938159 ] new vel:[ 7.07102283e-01 -1.92338190e-03  6.09008664e+00]
Frame 2 added with position [0.79909447 2.00190653 1.80282457] new vel:[ 7.07102283e-01 -1.92338190e-03  5.10908664e+00]
Frame 3 added with position [0.8698047  2.00171419 2.31373323] new vel:[ 7.07102283e-01 -1.92338190e-03  4.12808664e+00]
Frame 4 added with position [0.94051493 2.00

In [655]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[0])
release_frame.setPosition(result['release_position'])
C.view()
time.sleep(15)

### Clean up

In [656]:
del C
del bot